In [1]:
import os
import torch
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
import json
from PIL import Image
from tqdm import tqdm

# DSM Shot Detection
In this notebook we'll try to recreate DSM shot detection as outlined in [this paper](https://arxiv.org/pdf/1808.04234.pdf).

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Part 1: Adaptive Filtering

The first part of the DSM pipeline is adaptive filtering using SqueezeNet activations trained on ImageNet.

In [3]:
# First, load up the results of adaptive filtering from when they did it
items = []
with open('/app/data/dsm_cut_detection/data/data_list/cut_train_data_video.txt', 'r') as f:
    for line in f.readlines():
        boundary = line.strip().split(' ')
        if len(boundary) == 5:
            [_, path, start_frame, end_frame, label] = boundary
        else:
            [path, start_frame, end_frame, label] = boundary
        path = os.path.basename(path)
        start_frame = int(start_frame)
        end_frame = int(end_frame)
        label = int(label)
        items.append((path, start_frame, end_frame, label))

In [4]:
vids = {}
for i in items:
    key = i[0]
    if key not in vids:
        vids[key] = []
    vids[key].append((i[1], i[2], i[3]))

In [5]:
test_vid = items[0][0]

## SqueezeNet

In [6]:
squeezenet = models.squeezenet1_1(pretrained=True).to(device).eval()

/usr/local/lib/python3.5/dist-packages/torchvision/models/squeezenet.py:94: UserWarning: nn.init.kaiming_uniform is now deprecated in favor of nn.init.kaiming_uniform_.
  init.kaiming_uniform(m.weight.data)
/usr/local/lib/python3.5/dist-packages/torchvision/models/squeezenet.py:92: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  init.normal(m.weight.data, mean=0.0, std=0.01)


## Compute SqueezeNet embeddings

In [7]:
import random
import math
import numbers
import collections
import numpy as np
import torch
from PIL import Image, ImageOps
try:
    import accimage
except ImportError:
    accimage = None


class Compose(object):
    """Composes several transforms together.
    Args:
        transforms (list of ``Transform`` objects): list of transforms to compose.
    Example:
        >>> transforms.Compose([
        >>>     transforms.CenterCrop(10),
        >>>     transforms.ToTensor(),
        >>> ])
    """

    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, img):
        for t in self.transforms:
            img = t(img)
        return img

    def randomize_parameters(self):
        for t in self.transforms:
            t.randomize_parameters()


class ToTensor(object):
    """Convert a ``PIL.Image`` or ``numpy.ndarray`` to tensor.
    Converts a PIL.Image or numpy.ndarray (H x W x C) in the range
    [0, 255] to a torch.FloatTensor of shape (C x H x W) in the range [0.0, 1.0].
    """

    def __init__(self, norm_value=255):
        self.norm_value = norm_value

    def __call__(self, pic):
        """
        Args:
            pic (PIL.Image or numpy.ndarray): Image to be converted to tensor.
        Returns:
            Tensor: Converted image.
        """
        if isinstance(pic, np.ndarray):
            # handle numpy array
            img = torch.from_numpy(pic.transpose((2, 0, 1)))
            # backward compatibility
            return img.float().div(self.norm_value)

        if accimage is not None and isinstance(pic, accimage.Image):
            nppic = np.zeros([pic.channels, pic.height, pic.width], dtype=np.float32)
            pic.copyto(nppic)
            return torch.from_numpy(nppic)

        # handle PIL Image
        if pic.mode == 'I':
            img = torch.from_numpy(np.array(pic, np.int32, copy=False))
        elif pic.mode == 'I;16':
            img = torch.from_numpy(np.array(pic, np.int16, copy=False))
        else:
            img = torch.ByteTensor(torch.ByteStorage.from_buffer(pic.tobytes()))
        # PIL image mode: 1, L, P, I, F, RGB, YCbCr, RGBA, CMYK
        if pic.mode == 'YCbCr':
            nchannel = 3
        elif pic.mode == 'I;16':
            nchannel = 1
        else:
            nchannel = len(pic.mode)
        img = img.view(pic.size[1], pic.size[0], nchannel)
        # put it from HWC to CHW format
        # yikes, this transpose takes 80% of the loading time/CPU
        img = img.transpose(0, 1).transpose(0, 2).contiguous()
        if isinstance(img, torch.ByteTensor):
            return img.float().div(self.norm_value)
        else:
            return img

    def randomize_parameters(self):
        pass


class Normalize(object):
    """Normalize an tensor image with mean and standard deviation.
    Given mean: (R, G, B) and std: (R, G, B),
    will normalize each channel of the torch.*Tensor, i.e.
    channel = (channel - mean) / std
    Args:
        mean (sequence): Sequence of means for R, G, B channels respecitvely.
        std (sequence): Sequence of standard deviations for R, G, B channels
            respecitvely.
    """

    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, tensor):
        """
        Args:
            tensor (Tensor): Tensor image of size (C, H, W) to be normalized.
        Returns:
            Tensor: Normalized image.
        """
        # TODO: make efficient
        for t, m, s in zip(tensor, self.mean, self.std):
            t.sub_(m).div_(s)
        return tensor

    def randomize_parameters(self):
        pass


class Scale(object):
    """Rescale the input PIL.Image to the given size.
    Args:
        size (sequence or int): Desired output size. If size is a sequence like
            (w, h), output size will be matched to this. If size is an int,
            smaller edge of the image will be matched to this number.
            i.e, if height > width, then image will be rescaled to
            (size * height / width, size)
        interpolation (int, optional): Desired interpolation. Default is
            ``PIL.Image.BILINEAR``
    """

    def __init__(self, size, interpolation=Image.BILINEAR):
        assert isinstance(size, int) or (isinstance(size, collections.Iterable) and len(size) == 2)
        self.size = size
        self.interpolation = interpolation

    def __call__(self, img):
        """
        Args:
            img (PIL.Image): Image to be scaled.
        Returns:
            PIL.Image: Rescaled image.
        """
        if isinstance(self.size, int):
            w, h = img.size
            if (w <= h and w == self.size) or (h <= w and h == self.size):
                return img
            if w < h:
                ow = self.size
                oh = int(self.size * h / w)
                return img.resize((ow, oh), self.interpolation)
            else:
                oh = self.size
                ow = int(self.size * w / h)
                return img.resize((ow, oh), self.interpolation)
        else:
            return img.resize(self.size, self.interpolation)

    def randomize_parameters(self):
        pass

def get_mean(norm_value=255):
    return [114.7748 / norm_value, 107.7354 / norm_value, 99.4750 / norm_value]

def get_test_spatial_transform(opt):
    return Compose([Scale((opt.spatial_size,opt.spatial_size)),
                    ToTensor(opt.norm_value),
                    Normalize(get_mean(opt.norm_value), [1, 1, 1])])

In [8]:
def pil_loader(path):
    if not os.path.exists(path):
        return None
    try:
        with open(path, 'rb') as f:
            with Image.open(f) as img:
                return img.convert('RGB')
    except:
        return None

In [9]:
class Opt:
    def __init__(self):
        self.norm_value = 1
        self.spatial_size = 224

In [10]:
class VidDataset(Dataset):
    def __init__(self, path, frame_num):
        self.path = path
        self.frame_num = frame_num
        self.transform = get_test_spatial_transform(Opt())
    
    def __len__(self):
        return self.frame_num
    
    def __getitem__(self, idx):
        return self.transform(pil_loader(os.path.join(self.path, 'image_{}.jpg'.format(idx + 1))))

In [11]:
annotations_path = '/app/data/ClipShots/annotations/train.json'
frame_path = os.path.join('/app/data/ClipShots/frames/train', test_vid)

In [12]:
with open(annotations_path, 'r') as f:
    train_annotations = json.load(f)

In [13]:
test_vid_dataset = VidDataset(frame_path, int(train_annotations[test_vid]['frame_num']))

In [45]:
test_vid_dataloader = DataLoader(test_vid_dataset, shuffle=False, batch_size=8, num_workers=0)

In [46]:
squeezenet_features = []
squeezenet_classifications = []

In [47]:
for images in tqdm(test_vid_dataloader):
    images = images.to(device)
    squeezenet_classifications += squeezenet(images).detach().cpu().numpy().tolist()
    squeezenet_features += squeezenet.features(images).detach().cpu().numpy().tolist()





  0%|                                                                                                                                                                                                                | 0/605 [00:00<?, ?it/s]



  0%|▎                                                                                                                                                                                                       | 1/605 [00:00<01:25,  7.05it/s]



  0%|▋                                                                                                                                                                                                       | 2/605 [00:00<01:21,  7.37it/s]



  0%|▉                                                                                                                                                                                                       | 3/605 [00:00<01:19,  7.61it/s]



  1%|█▎                         

  8%|████████████████▊                                                                                                                                                                                      | 51/605 [00:05<00:53, 10.34it/s]



  9%|█████████████████▍                                                                                                                                                                                     | 53/605 [00:06<01:57,  4.70it/s]



  9%|█████████████████▊                                                                                                                                                                                     | 54/605 [00:06<01:50,  4.98it/s]



  9%|██████████████████                                                                                                                                                                                     | 55/605 [00:06<01:34,  5.84it/s]



  9%|██████████████████▋            

 18%|███████████████████████████████████                                                                                                                                                                   | 107/605 [00:13<00:53,  9.30it/s]



 18%|███████████████████████████████████▋                                                                                                                                                                  | 109/605 [00:14<02:15,  3.67it/s]



 18%|████████████████████████████████████▎                                                                                                                                                                 | 111/605 [00:14<01:46,  4.65it/s]



 19%|████████████████████████████████████▋                                                                                                                                                                 | 112/605 [00:14<01:29,  5.50it/s]



 19%|███████████████████████████████

 25%|██████████████████████████████████████████████████                                                                                                                                                    | 153/605 [00:20<00:51,  8.73it/s]



 26%|██████████████████████████████████████████████████▋                                                                                                                                                   | 155/605 [00:20<00:49,  9.12it/s]



 26%|███████████████████████████████████████████████████                                                                                                                                                   | 156/605 [00:21<00:48,  9.27it/s]



 26%|███████████████████████████████████████████████████▋                                                                                                                                                  | 158/605 [00:21<00:47,  9.32it/s]



 26%|███████████████████████████████

 35%|████████████████████████████████████████████████████████████████████▍                                                                                                                                 | 209/605 [00:28<00:33, 11.72it/s]



 35%|█████████████████████████████████████████████████████████████████████                                                                                                                                 | 211/605 [00:28<00:33, 11.77it/s]



 35%|█████████████████████████████████████████████████████████████████████▋                                                                                                                                | 213/605 [00:28<00:32, 11.93it/s]



 36%|██████████████████████████████████████████████████████████████████████▎                                                                                                                               | 215/605 [00:28<00:32, 12.02it/s]



 36%|███████████████████████████████

...

 60%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏                                                                              | 364/605 [00:54<00:24,  9.92it/s]



 60%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                                                                              | 366/605 [00:54<00:26,  9.18it/s]



 61%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                                                                             | 368/605 [00:54<00:24,  9.53it/s]



 61%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                                                                             | 370/605 [00:54<00:25,  9.19it/s]



 61%|███████████████████████████████

 69%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                                                              | 415/605 [00:59<00:16, 11.84it/s]



 69%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                                                             | 417/605 [00:59<00:15, 11.96it/s]



 69%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏                                                            | 419/605 [00:59<00:15, 12.01it/s]



 70%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                                                            | 421/605 [00:59<00:15, 11.89it/s]



 70%|███████████████████████████████

 79%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                                         | 479/605 [01:09<00:10, 11.57it/s]



 80%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                                        | 481/605 [01:10<00:10, 11.77it/s]



 80%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                                        | 483/605 [01:10<00:10, 11.87it/s]



 80%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                       | 485/605 [01:10<00:10, 11.91it/s]



 80%|███████████████████████████████

 89%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                     | 539/605 [01:15<00:05, 11.49it/s]



 89%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                     | 541/605 [01:21<01:06,  1.03s/it]



 90%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                    | 543/605 [01:21<00:46,  1.34it/s]



 90%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                   | 545/605 [01:22<00:32,  1.82it/s]



 90%|███████████████████████████████

 98%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋   | 595/605 [01:27<00:01,  9.55it/s]



 99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████   | 596/605 [01:27<00:00,  9.67it/s]



 99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋  | 598/605 [01:27<00:00,  9.82it/s]



 99%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎ | 600/605 [01:27<00:00,  9.94it/s]



100%|███████████████████████████████